In [2]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [3]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [4]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [17]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_swta_t')),
}

In [18]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [19]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,100,37.59,23.14
1,H-UNet-SWTA-T,0,5,100,38.85,24.11
2,H-UNet-SWTA-T,0,10,100,40.69,25.55
3,H-UNet-SWTA-T,0,20,100,41.58,26.25
4,H-UNet-SWTA-T,0,50,100,41.81,26.43
5,H-UNet-SWTA-T,0,75,100,41.85,26.46
6,H-UNet-SWTA-T,0,100,100,41.87,26.48


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                Mean CI 0.9%    Mean CI 0.9%
Model         Inv Temp Regime                               
H-UNet-SWTA-T 1        100     37.59     NaN   23.14     NaN
              5        100     38.85     NaN   24.11     NaN
              10       100     40.69     NaN   25.55     NaN
              20       100     41.58     NaN   26.25     NaN
              50       100     41.81     NaN   26.43     NaN
              75       100     41.85     NaN   26.46     NaN
              100      100     41.87     NaN   26.48     NaN

In [11]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_unet_swta_t')),
}

In [12]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [13]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,1,68.56,52.16
1,H-UNet-SWTA-T,0,1,2,72.09,56.36
2,H-UNet-SWTA-T,0,1,5,75.92,61.18
3,H-UNet-SWTA-T,0,1,10,81.47,68.72
4,H-UNet-SWTA-T,0,1,20,81.90,69.35
...,...,...,...,...,...,...
345,H-UNet-SWTA-T,9,100,1,68.90,52.55
346,H-UNet-SWTA-T,9,100,2,67.77,51.25
347,H-UNet-SWTA-T,9,100,5,74.42,59.26
348,H-UNet-SWTA-T,9,100,10,77.89,63.79


Dice           Jaccard          
                                 Mean   CI 0.9%    Mean   CI 0.9%
Model         Inv Temp Regime                                    
H-UNet-SWTA-T 1        1       69.548  1.215891  53.354  1.467645
                       2       70.397  1.020861  54.343  1.216367
                       5       76.151  0.998454  61.514  1.309018
                       10      80.347  1.192206  67.197  1.664618
                       20      83.248  0.897270  71.331  1.313004
              5        1       69.387  1.122189  53.160  1.353242
                       2       70.583  1.245158  54.582  1.496884
                       5       76.432  0.891869  61.877  1.162379
                       10      79.863  0.887621  66.499  1.230385
                       20      82.959  0.717113  70.898  1.049360
              10       1       69.223  1.081086  52.962  1.303161
                       2       70.947  1.082406  55.005  1.291819
                       5       76.115  0.975241  61.470  1.269745
                       10      80.686  0.998373  67.661  1.399422
                       20      84.180  0.685501  72.697  1.021967
              20       1       69.515  1.111732  53.306  1.337037
                       2       71.023  1.409415  55.122  1.709910
                       5       76.014  1.053813  61.341  1.375073
                       10      79.711  0.804653  66.285  1.112092
                       20      83.990  0.772396  72.422  1.146995
              50       1       69.650  1.068405  53.463  1.288372
                       2       70.868  1.129622  54.915  1.351494
                       5       75.578  0.924106  60.768  1.203304
                       10      80.117  0.800758  66.847  1.109783
                       20      84.327  0.805715  72.923  1.208521
              75       1       69.253  0.988420  52.995  1.186579
                       2       70.585  1.174258  54.579  1.408982
                       5       76.077  1.126424  61.428  1.485568
                       10      80.563  1.161436  67.498  1.616119
                       20      84.040  0.434328  72.476  0.645080
              100      1       69.389  1.056743  53.155  1.279185
                       2       70.963  1.382009  55.048  1.678531
                       5       75.681  1.210225  60.922  1.582625
                       10      79.758  0.889401  66.357  1.228492
                       20      83.530  0.806403  71.742  1.182614

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [14]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'UNet': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('unet')),
    'EM': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('em_unet')),
    #'UAMT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('uamt_unet')),
    #'CPS': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('cps_unet')),
    #'URPC': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('urpc_unet')),
}

In [15]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [16]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,EM,0,1,1,67.89,51.39
1,EM,0,1,2,67.80,51.28
2,EM,0,1,5,71.81,56.01
3,EM,0,1,10,79.57,66.06
4,EM,0,1,20,79.95,66.60
5,EM,1,1,1,69.24,52.95
6,EM,1,1,2,68.65,52.27
7,EM,1,1,5,73.77,58.44
8,EM,1,1,10,67.77,51.25
9,EM,1,1,20,67.77,51.25


Dice           Jaccard          
                         Mean   CI 0.9%    Mean   CI 0.9%
Model Inv Temp Regime                                    
EM    1        1       68.468  0.743299  52.070  0.878361
               2       68.849  0.999877  52.521  1.194260
               5       73.278  1.831446  57.921  2.250502
               10      72.974  3.076267  57.724  3.839382
               20      73.330  3.318815  58.215  4.183189